In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_excel("mlb odds 2021.xlsx")
pitchers = pd.read_excel("Pitchers.xlsx")

In [4]:
#https://towardsdatascience.com/can-we-beat-the-bookmaker-with-machine-learning-45e3b30fc921
teams = list(set(df['Home Team'].values)) # creating a list containing every team
n_teams = len(teams) # number of teams
teamToIdx = {t: i for i, t in enumerate(teams)} # our team vocabulary

# assigning the games' home teams their corresponding team id
homeId = [teamToIdx[id] for id in list(df['Home Team'].values)]
df['Home_Id'] = homeId #creating a new column for the home team id

# assigning the games' Visitors teams their corresponding team id
visitorsId = [teamToIdx[id] for id in list(df['Away Team'].values)]
df['Away_Id'] = visitorsId # creating a new row for the away team id

pitchers = list(set(pitchers['Home Pitcher'].values)) # creating a list containing every team
n_picters = len(pitchers) # number of teams
pitcherToIdx = {t: i for i, t in enumerate(pitchers)} # our team vocabulary

# assigning the games' home teams their corresponding team id
homePitcherId = [pitcherToIdx[id] for id in list(df['Home Pitcher'].values)]
df['HomePitcher_Id'] = homePitcherId #creating a new column for the home team id

# assigning the games' Visitors teams their corresponding team id
visitorsPitcherId = [pitcherToIdx[id] for id in list(df['Away Pitcher'].values)]
df['AwayPitcher_Id'] = visitorsPitcherId # creating a new row for the away team id

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               2644 non-null   int64  
 1   Home Team          2644 non-null   object 
 2   Home Pitcher       2644 non-null   object 
 3   Home Score         2644 non-null   int64  
 4   Home Odds          2644 non-null   int64  
 5   Home RunLine       2644 non-null   float64
 6   Home RunLine Odds  2644 non-null   int64  
 7   OpenOU             2644 non-null   float64
 8   Under Odds         2644 non-null   int64  
 9   Away Team          2644 non-null   object 
 10  Away Pitcher       2644 non-null   object 
 11  Away Score         2644 non-null   int64  
 12  Away Odds          2644 non-null   int64  
 13  Away RunLine       2644 non-null   float64
 14  Away RunLine Odds  2644 non-null   int64  
 15  Over Odds          2644 non-null   int64  
 16  Winner             2644 

In [6]:
X = df.drop(columns=['Winner','Home Team','Home Pitcher',"Home Score", "Away Team", "Away Score", "Away Pitcher"])
Y = df['Winner']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)


In [8]:
rf = RandomForestClassifier(n_estimators = 400)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=400)

In [12]:
prediction = rf.predict(X_test)
accuracy = accuracy_score(y_test, prediction)
print("Model is {:.2%} accurate on test data".format(accuracy))

Model is 53.88% accurate on test data


In [13]:
def decimal_odds(amount):
  if amount > 0:  #If postive
    decimal = amount/100 + 1
  else:           #If negative
    decimal = 1 - 100/amount 
  return decimal

In [14]:
units = 50


home_odds = np.array(X_test.loc[:,'Home Odds'])
visitors_odds = np.array(X_test.loc[:,'Away Odds'])
game_outcome = np.array(y_test)

total_bets_placed = 0

possible_winnings = np.zeros(len(home_odds))

for i in range(len(possible_winnings)):
  total_bets_placed = total_bets_placed + units

  if  (game_outcome[i]+prediction[i]) == 2: #Home team wins and prediction correct
    possible_winnings[i] = units * decimal_odds(home_odds[i])
  elif (game_outcome[i]+prediction[i]) == 0: #Visitors team wins and prediction correct
    possible_winnings[i] = units * decimal_odds(visitors_odds[i])
  else:
    possible_winnings[i] = 0


winnings = sum(possible_winnings) - total_bets_placed

print("If we were to place ${:.2f} on every test bet and win {:.2%} of the time, we would win ${:.2f}".format(units,accuracy,winnings))  

If we were to place $50.00 on every test bet and win 53.88% of the time, we would win $-1921.74
